In [6]:
print('Validate nvidia-smi (software managemtn interface):')
!nvidia-smi
print()
print('Validate nvcc --version:')
!nvcc --version

Validate nvidia-smi (software managemtn interface):
Mon Oct 16 06:52:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   34C    P0    12W /  N/A |      5MiB /  3910MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                    

In [12]:
# Below is from previous project 2: Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "/home/peter/SVU/SVU_CS512_DL/00_kaggle_cat_dog/PetImages"
CATEGORIES = ["Dog", "Cat"] 
IMG_SIZE = 50
training_data = []
def create_training_data():
    for category in CATEGORIES:  # do dogs and cats
        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat
        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))
create_training_data()
print(len(training_data))

import random
random.shuffle(training_data)
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # -1: keep all rows with image size, 1: grayscale
print('X.shape:', X.shape)
y = np.array(y) # convert array into np.array (tensor) for tensorflow 2.0
import pickle
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

100%|███████████████████████████████████| 12501/12501 [00:08<00:00, 1482.32it/s]


24946
X.shape: (24946, 50, 50, 1)


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle

#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session (config=tf.ConfigProto(gpu_options=gpu_options))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
sess = InteractiveSession(config=config)

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y) # can we do convert np.array twice?

model = Sequential()
# X.shape = (24946, 50, 50, 1)
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=30, epochs = 3, validation_split=0.3)

2023-10-16 11:22:53.181577: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-16 11:22:53.181599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: knode1
2023-10-16 11:22:53.181603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: knode1
2023-10-16 11:22:53.181770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.199.2
2023-10-16 11:22:53.181814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.199.2
2023-10-16 11:22:53.181819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.199.2


Epoch 1/3


2023-10-16 11:22:54.079947: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 174620000 exceeds 10% of free system memory.
2023-10-16 11:22:55.062885: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70778880 exceeds 10% of free system memory.


  1/583 [..............................] - ETA: 9:52 - loss: 0.7035 - accuracy: 0.4333

2023-10-16 11:22:55.399629: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70778880 exceeds 10% of free system memory.
2023-10-16 11:22:55.489508: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70778880 exceeds 10% of free system memory.


  2/583 [..............................] - ETA: 4:07 - loss: 2.4449 - accuracy: 0.4500

2023-10-16 11:22:55.835338: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 70778880 exceeds 10% of free system memory.


583/583 [==============================] - 255s 436ms/step - loss: 0.6774 - accuracy: 0.5815 - val_loss: 0.6806 - val_accuracy: 0.5601
Epoch 2/3
583/583 [==============================] - 254s 435ms/step - loss: 0.6447 - accuracy: 0.6382 - val_loss: 0.6385 - val_accuracy: 0.6367
Epoch 3/3
583/583 [==============================] - 258s 442ms/step - loss: 0.6232 - accuracy: 0.6612 - val_loss: 0.6183 - val_accuracy: 0.6554


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

# Add the tensorboard 
NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))
print ('NAME:', NAME)
tensorboard = TensorBoard (log_dir='log/{}'.format(NAME))
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session (config=tf.ConfigProto(gpu_options=gpu_options))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
sess = InteractiveSession(config=config)

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y) # can we do convert np.array twice?

model = Sequential()
# X.shape = (24946, 50, 50, 1)
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# Add Tebsorboard object into callback. We change the epochs = 10
model.fit(X, y, batch_size=30, epochs = 10, validation_split=0.3, callbacks=[tensorboard])

NAME: Cats-vs-dog-cnn-64x2-1697484886


/home/peter/anaconda3/envs/tf/lib/python3.11/site-packages/tensorflow/python/client/session.py:1793: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1/10
583/583 [==============================] - 252s 431ms/step - loss: 0.6655 - accuracy: 0.6042 - val_loss: 0.6413 - val_accuracy: 0.6269
Epoch 2/10
583/583 [==============================] - 249s 426ms/step - loss: 0.6032 - accuracy: 0.6738 - val_loss: 0.5668 - val_accuracy: 0.7104
Epoch 3/10
583/583 [==============================] - 233s 400ms/step - loss: 0.5539 - accuracy: 0.7208 - val_loss: 0.5496 - val_accuracy: 0.7242
Epoch 4/10
583/583 [==============================] - 234s 401ms/step - loss: 0.5200 - accuracy: 0.7437 - val_loss: 0.5069 - val_accuracy: 0.7602
Epoch 5/10
583/583 [==============================] - 234s 401ms/step - loss: 0.4852 - accuracy: 0.7687 - val_loss: 0.4809 - val_accuracy: 0.7773
Epoch 6/10
583/583 [==============================] - 234s 401ms/step - loss: 0.4621 - accuracy: 0.7835 - val_loss: 0.5094 - val_accuracy: 0.7664
Epoch 7/10
583/583 [==============================] - 233s 400ms/step - loss: 0.4518 - accuracy: 0.7893 - val_loss: 0.4809 -

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

# Add the tensorboard 
NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))
print ('NAME:', NAME)
tensorboard = TensorBoard (log_dir='log/{}'.format(NAME))
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session (config=tf.ConfigProto(gpu_options=gpu_options))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
sess = InteractiveSession(config=config)

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y) # can we do convert np.array twice?

model = Sequential()
# X.shape = (24946, 50, 50, 1)
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# Remove the Dense layer.
#model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# Change Epochs from 10 to 20.
model.fit(X, y, batch_size=30, epochs = 20, validation_split=0.3, callbacks=[tensorboard])

NAME: Cats-vs-dog-cnn-64x2-1697502452
Epoch 1/20
583/583 [==============================] - 251s 429ms/step - loss: 0.6286 - accuracy: 0.6374 - val_loss: 0.6133 - val_accuracy: 0.6648
Epoch 2/20
583/583 [==============================] - 279s 479ms/step - loss: 0.5469 - accuracy: 0.7251 - val_loss: 0.5300 - val_accuracy: 0.7318
Epoch 3/20
583/583 [==============================] - 222s 380ms/step - loss: 0.5025 - accuracy: 0.7583 - val_loss: 0.4986 - val_accuracy: 0.7610
Epoch 4/20
583/583 [==============================] - 221s 380ms/step - loss: 0.4779 - accuracy: 0.7718 - val_loss: 0.4710 - val_accuracy: 0.7805
Epoch 5/20
583/583 [==============================] - 222s 380ms/step - loss: 0.4556 - accuracy: 0.7879 - val_loss: 0.4831 - val_accuracy: 0.7726
Epoch 6/20
583/583 [==============================] - 221s 379ms/step - loss: 0.4305 - accuracy: 0.7993 - val_loss: 0.4935 - val_accuracy: 0.7718
Epoch 7/20
583/583 [==============================] - 222s 380ms/step - loss: 0.4025 -